In [ ]:
!pip install cpufeature
!pip install triton==0.2.3
!DS_BUILD_CPU_ADAM=1 DS_BUILD_SPARSE_ATTN=1 pip install deepspeed==0.3.7
!pip install transformers==3.5.0

In [ ]:
!pip install --no-cache-dir pytelegrambotapi

In [ ]:
import telebot
import uuid
import os
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import string
from copy import copy
import random
import re

from transformers import GPT2LMHeadModel, GPT2Tokenizer


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#!mkdir /home/jovyan/devices/quests/generated_quests

In [ ]:
token = '' #here should be token
bot = telebot.TeleBot(token)

/home/user/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
MODEL_PATH = "/home/jovyan/devices/quests/hf"
TOKENIZER_PATH = "/home/jovyan/devices/quests/hf"
QUEST_DIRECTORY = '/home/jovyan/devices/quests/generated_quests'

In [ ]:
gpt_model = GPT2LMHeadModel.from_pretrained(MODEL_PATH)
tokenizer = GPT2Tokenizer.from_pretrained(TOKENIZER_PATH)

In [ ]:
gpt_model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [ ]:
#os.mkdir(QUEST_DIRECTORY)

In [ ]:
def calc_perplexity(input_idx, gpt2model):
    loss = gpt_model(input_idx, labels=input_idx)[0]
    return loss.exp()  

In [ ]:
ru_stopwords = stopwords.words('russian')
stemmer = SnowballStemmer('russian')
punctuation = string.punctuation + '«»'

def get_stem_set(text):
    tokens = [token for token in wordpunct_tokenize(text) if token not in ru_stopwords and token not in punctuation]
    return set([stemmer.stem(token) for token in tokens])
    

In [ ]:
class NextFragmentCandidate:
    def __init__(self, text:str, gpt2model, tokenizer, pretext='', posttext=''):
        self.text = text
        joined_text = pretext+text+posttext
        self.input_idx = tokenizer.encode(joined_text, return_tensors='pt').cuda()
        self.perplexity = calc_perplexity(self.input_idx, gpt2model)
        
    def __len__(self):
        return self.input_idx.shape[-1]    

In [ ]:
class Story: 
    def __init__(self, beginning:str, middle_fragments:list, endings:list, gpt2model, tokenizer, beginning_stem_set=None, max_score2suggest=5000):
        self.beginning = beginning
        self.beginning_stem_set = beginning_stem_set if beginning_stem_set else get_stem_set(beginning)
        
        self.pretext = 'BOF'
        self.gpt2model = gpt2model
        self.tokenizer = tokenizer
        self.threshold = max_score2suggest
        
        self.middle_part_fragments = self.__get_fragments_field(middle_fragments)
        self.endings = self.__get_fragments_field(endings)
        
    def score_beginning_suitability(self, text, text_stem_set):
        if text.lower().strip() == self.beginning.lower().strip():
            return 1.1
        elif not text_stem_set or not self.beginning_stem_set:
            return 0
        else: 
            return len(self.beginning_stem_set.intersection(text_stem_set))/min(len(self.beginning_stem_set), len(text_stem_set))
      
    def __get_fragments_field(self, fragments:list):
        if all([isinstance(fragment, NextFragmentCandidate) for fragment in fragments]):
            return copy(fragments)
        elif all([isinstance(fragment, str) for fragment in fragments]):
            return [NextFragmentCandidate(fragment, self.gpt2model, self.tokenizer, pretext=self.pretext) for fragment in fragments]
        else:
            raise ValueError('fragments should be str or NextFragmentCandidate objects')
    
    def __score_next_fragment_candidate(self, text, next_fragment_candidate:NextFragmentCandidate, pretext='BOF', intertext='BOF', posttext=''):
        joined_text = pretext + text + intertext + next_fragment_candidate.text + posttext
        input_idx = self.tokenizer.encode(joined_text, return_tensors='pt').cuda()
        joined_text_len = input_idx.shape[-1]
        joined_text_perplexity = calc_perplexity(input_idx, self.gpt2model)
        score = (joined_text_perplexity-next_fragment_candidate.perplexity)/(joined_text_len-len(next_fragment_candidate))
        return float(score)
    
    def remove_candidate(self, text, next_fragment_type):
        if next_fragment_type == 'standart':
            self.middle_part_fragments = [candidate for candidate in self.middle_part_fragments if candidate.text != text]
        elif next_fragment_type == 'ending':
            self.endings = [candidate for candidate in self.endings if candidate.text != text]
        else:
            raise ValueError('next candidate type should be "standart" or "ending"')
        
    
    def suggest_best_next(self, previous_fragment, next_fragment_type, remove_candidate=True):
        if next_fragment_type == 'standart':
            next_fragment_candidates = self.middle_part_fragments
            is_ending = False
        elif next_fragment_type == 'ending':
            next_fragment_candidates = self.endings
            is_ending = True
        else:
            next_fragment_candidates = None
        if next_fragment_candidates:
            scored_fragments = [(next_fragment_candidate.text, self.__score_next_fragment_candidate(previous_fragment, next_fragment_candidate)) for next_fragment_candidate in next_fragment_candidates]
            sorted_scored_fragments = sorted(scored_fragments, key = lambda x: x[1])
            best_candidate_data = sorted_scored_fragments[0]
            if best_candidate_data[1] <= self.threshold:
                best_next_text = best_candidate_data[0]
                if remove_candidate:
                    self.remove_candidate(best_next_text, next_fragment_type)
                return best_next_text, is_ending
            else:
                return None, False
        else:
            return None, False
    
    def copy(self):
        return Story(copy(self.beginning), copy(self.middle_part_fragments), copy(self.endings), self.gpt2model, self.tokenizer, copy(self.beginning_stem_set), copy(self.threshold))

In [ ]:
stories_data = [
    [
        'Вы идете по зимнему лесу и слышите детский крик.',
        [
             'Вы выходите на поляну и видите недостроенный стул из снега, возле которого лежит грустный плюшевый мишка. Хотите достроить?',
             'Вы подошли к выходу из леса и еще можете успеть на последний автобус в город. Но вы так и не выяснили, что произошло с ребенком. Что будете делать?',
             'Вы выходите к поляне, надеетесь, что идти по ней будет легче. Делаете шаг вперед и проваливаетесь по пояс. Развернетесь?',
             'Вы слышите тот же детский голос снова. Но теперь ребенок не кричит, а репетирует «Я помню чудное мгновенье». Но почему он делает это в лесу?',
             'Вы обнаруживаете следы на снегу, размер ноги определенно не как у ребенка. Посмотрите, куда они ведут.',
             'Вы совсем забыли, откуда пришли. Может, найти дорогу назад важнее, чем понять, откуда доносились звуки?'
        ],
        [
            'Между деревьев вы обнаруживаете колонку, из которой доносится тот же детский крик. Вы выключаете ее, чтобы звуки больше никого не сбили с толку.',
            'Теперь голос доносится сверху, причем очень близко. Под вашими ногами валяется ботинок, ребенок сидит наверху и не хочет слезать в сугроб. Вы подаете ему обувь, он благодарит вас и говорит, что слезет, когда захочет.'
        ]
    ]
]

In [ ]:
stories = [Story(*story_data, gpt_model, tokenizer) for story_data in stories_data]

In [ ]:
class UserStoryOrganizer:
    def __init__(self, story, min_steps_from_previous=3, min_story_len=8):
        self.story = story.copy()
        self.steps_from_beginning = 0
        self.steps_from_previous = 0
        
        self.min_steps_from_previous = min_steps_from_previous
        self.min_story_len = min_story_len
        
    def __get_best_next_human_written_types(self, ignore_num_steps_from_previous=False):
        if self.steps_from_previous <= self.min_steps_from_previous or ignore_num_steps_from_previous:
            if self.steps_from_beginning <= self.min_story_len:
                return ['standart']
            else:
                return ['ending', 'standart']
        else:
            return []
        
    def increase_steps_counters(self):
        self.steps_from_beginning += 1
        self.steps_from_previous += 1
        
    def suggest_next_fragment(self, previous_text, increase_counters=False, ignore_num_steps_from_previous=False):
        next_fragment = None
        is_ending = False
        suitable_next_types = self.__get_best_next_human_written_types(ignore_num_steps_from_previous)
        if suitable_next_types:
            for next_fragment_type in suitable_next_types:
                next_fragment, is_ending = self.story.suggest_best_next(previous_text, next_fragment_type)
                if next_fragment:
                    break
        if increase_counters:
            self.increase_steps_counters()
        if next_fragment:
            self.steps_from_previous = 0
        return next_fragment, is_ending
    
    def time_for_human_written(self, increase_counter=True):
        if increase_counter:
            self.increase_steps_counters()
        return self.steps_from_previous >= self.min_steps_from_previous

In [ ]:
class StoryFragmentsManager:
    def __init__(self, stories:list, starting_story_treshold=0.7):
        self.stories = stories
        self.__user_story_organizers = {}
        self.starting_story_treshold = starting_story_treshold
        
    def get_random_story_beginning(self):
        story = random.choice(self.stories)
        return story.beginning
    
    def set_story(self, session_id, story):
        self.__user_story_organizers[session_id] = UserStoryOrganizer(story)
        
    def forget_story(self, session_id):
        if session_id in self.__user_story_organizers:
            del self.__user_story_organizers[session_id]
    
    def analyze_user_input_first_fragment(self, session_id, user_input):
        suitable_story = None
        suitable_story_score = 0
        input_stem_set = get_stem_set(user_input)
        for story in self.stories:
            story_score = story.score_beginning_suitability(user_input, input_stem_set)
            if story_score > suitable_story_score and story_score > self.starting_story_treshold:
                suitable_story = story
                suitable_story_score = story_score
                if story_score > 1:
                    break
        if suitable_story:
            self.set_story(session_id, suitable_story)
            
    def suggest_next_fragment(self, session_id, previous_fragment):
        if session_id not in self.__user_story_organizers:
            return None, False
        else:
            text, is_ending = self.__user_story_organizers[session_id].suggest_next_fragment(previous_fragment)
            return text, is_ending
        
    def time_for_human_written(self, session_id, increase_counters=True):
        return session_id in self.__user_story_organizers and self.__user_story_organizers[session_id].time_for_human_written(increase_counters)

In [ ]:
class ChatSessionsFilesOrganizer:
    
    def __init__(self, directory, extension='.txt', encoding='utf-8'):
        self.__chat_id2current_quest_file = {}
        self._directory = directory
        self._extension = extension
        self._encoding = encoding
    
    def create_new_quest_file(self, session_id, start_text=''):
        quest_id = uuid.uuid1().hex
        file_name = quest_id + self._extension
        self.__chat_id2current_quest_file[session_id] = file_name
        file_path = os.path.join(self._directory, file_name)
        with open(file_path, 'w', encoding=self._encoding) as f:
            if start_text:
                f.write(start_text)
        
    def get_current_quest_file_path(self, session_id):
        if session_id not in self.__chat_id2current_quest_file:
            self.create_new_quest_file(session_id)
        current_quest_file = self.__chat_id2current_quest_file[session_id]
        current_quest_path = os.path.join(self._directory, current_quest_file)
        return current_quest_path
    
    def get_current_quest_text(self, session_id):
        quest_file_path = self.get_current_quest_file_path(session_id)
        with open(quest_file_path, encoding=self._encoding) as f:
            text = f.read()
        return text
    
    def add_next_part(self, session_id, new_fragment, separator='\n'):
        file_path = self.get_current_quest_file_path(session_id)
        with open(file_path, 'a', encoding=self._encoding) as f:
            f.write(separator + new_fragment)
            
    def get_last_fragment(self, session_id, fragments_separator):
        text = self.get_current_quest_text(session_id)
        last_fragment = text.split(fragments_separator)[-1]
        return last_fragment
    
    def unlink_file(self, session_id):
        if session_id in self.__chat_id2current_quest_file:
            del self.__chat_id2current_quest_file[session_id]
 
 
class GptGenerator:
    def __init__(self, gpt_model, tokenizer, max_pretext_length = 2048, default_not_greedy_temperature=1.5, default_greedy_temperature=1):
        self.gpt_model = gpt_model
        self.tokenizer = tokenizer
        self.__max_pretext_length = max_pretext_length
        self.tokenizer.add_special_tokens({"bos_token": "<s>"})
        self.tokenizer.add_special_tokens({"eos_token": "</s>"})      
        
        self.default_greedy_temperature = default_greedy_temperature
        self.default_not_greedy_temperature = default_not_greedy_temperature
    
     
    def get_next_steps(self, text, max_length=200, greedy=False, temperature=None, num_beams=None):
        pretext_length = len(text)
        input_idx = self.tokenizer.encode(text, return_tensors='pt').cuda()
        if input_idx.shape[1] > self.__max_pretext_length:
            print('Обрезаем претекст')
            input_idx = input_idx[:, -self.__max_pretext_length:]
            used_pretext = tokenizer.decode(input_idx[0])
            pretext_length = len(used_pretext)
        if temperature == None:
            if greedy:
                temperature = self.default_greedy_temperature
            else:
                temperature = self.default_not_greedy_temperature
        if greedy:
             encoded_new_text = self.gpt_model.generate(input_idx, max_length=pretext_length+max_length, temperature=temperature, repetition_penalty=5.0, top_k=5, top_p=0.95, do_sample=True)[0]
        else:
            encoded_new_text = self.gpt_model.generate(input_idx, do_sample=True, max_length=pretext_length+max_length, num_beams=num_beams, temperature=temperature)[0]
        new_text = self.tokenizer.decode(encoded_new_text)
        return new_text[pretext_length:]


help_message_text =  'Я бот, я могу сгенерировать квест! Введите команду /start, чтобы начать новый квест.'
start_message_text = 'Нажмите на кнопку с началом квеста или придумайте свое'
suggest_new_quest_button_text = 'Начать новый квест'
    
class QuestGenerator:
    def __init__(self, model, tokenizer, quests_directory, stories, starting_story_treshold=0.7, end_button_text=suggest_new_quest_button_text, ending_text='Квест завершен!'):
        self.__gpt2generator = GptGenerator(model, tokenizer)
        self.__human_written_stories_manager = StoryFragmentsManager(stories, starting_story_treshold)
        self.__files_organizer = ChatSessionsFilesOrganizer(quests_directory)
        self.__start_quest_tag = 'BOQ'
        self.__end_quest_tag = 'EOQ'
        self.__start_fragment_tag = 'BOF'
        self.__button_tag = 'BUTTON'
        self.__input_tag = 'INPUT'
        self.separator = '\n'
        self.end_button_text = end_button_text
        self.ending_text = ending_text
        
        self.max_button_len = 30
        self.max_fragment_len = 100
    
    def new_quest(self, session_id):
        self.__files_organizer.create_new_quest_file(session_id)
        
    def suggest_beginning(self):
        return self.__human_written_stories_manager.get_random_story_beginning()
        
    def save_first_fragment(self, session_id, text):
        self.__human_written_stories_manager.analyze_user_input_first_fragment(session_id, text)
        self.save_fragment(session_id, text)
    
    def save_fragment(self, session_id, text):
        new_fragment = self.separator.join([self.__start_fragment_tag, text])
        self.__files_organizer.add_next_part(session_id, new_fragment)
        
    def generate_buttons(self, session_id):
        text = self.__files_organizer.get_current_quest_text(session_id)
        text  = self.__make_text_end_with(text, self.__button_tag)
        generated_next_steps = self.__gpt2generator.get_next_steps(text)
        button_texts = self.__clean_and_save_button_texts(session_id, generated_next_steps) 
        return button_texts
       
    def save_input(self, session_id, user_input):
        new_fragment = self.separator.join([self.__input_tag, user_input])
        self.__files_organizer.add_next_part(session_id, new_fragment)
        
    def generate_fragment(self, session_id):
        is_ending = False
        fragment_text = None
        buttons_plain_text = ''
        if self.__human_written_stories_manager.time_for_human_written(session_id):
            full_previous_fragment = self.__files_organizer.get_last_fragment(session_id, fragments_separator=self.__start_fragment_tag)
            previous_fragment = full_previous_fragment.split(self.__button_tag)[0] + full_previous_fragment.split(self.__input_tag)[-1]
            fragment_text, is_ending = self.__human_written_stories_manager.suggest_next_fragment(session_id, previous_fragment)
        if not fragment_text:
            text = self.__files_organizer.get_current_quest_text(session_id)
            text = self.__make_text_end_with(text, self.__start_fragment_tag)
            generated_next_steps = self.__gpt2generator.get_next_steps(text)  
            generated_next_steps = generated_next_steps.split(self.__start_fragment_tag)[0]
            generated_next_steps = generated_next_steps.split(self.__input_tag)[0]
            
            if self.__end_quest_tag in generated_next_steps:
                fragment_text = generated_next_steps.split(self.__end_quest_tag)[0] 
                fragment_text = generated_next_steps.split(self.__button_tag)[0] 
                is_ending = True
            elif self.__button_tag not in generated_next_steps:
                fragment_text = self.__strip_text(generated_next_steps, self.max_fragment_len)
            else:
                parts = generated_next_steps.split(self.__button_tag)
                fragment_text = parts[0]
                buttons_plain_text = self.__button_tag.join(parts[1:])
                fragment_text = self.__strip_text(fragment_text, self.max_fragment_len)
                self.save_fragment(session_id, fragment_text)
                #if buttons are generated and we can be sure that at least one button text is finished
        if is_ending:
            self.__human_written_stories_manager.forget_story(session_id)
            self.__files_organizer.unlink_file(session_id)
            button_texts = [self.end_button_text]
            fragment_text = fragment_text.split(self.__end_quest_tag)[0]
            fragment_text = ' '.join([fragment_text, self.ending_text])
        elif buttons_plain_text:
            button_texts = self.__clean_and_save_button_texts(session_id, buttons_plain_text)
        else:
            button_texts = []
        return fragment_text, button_texts, is_ending
            
    def __strip_text(self, text, max_len):
        stripped_text = text
        if len(text) > max_len:
            #ToDo: может генериться текст без точек,
            #надо будет сюда добавить дробление подозрительно больших предложений другим способом
            sents = [sent for sent in sent_tokenize(text) if sent.strip()]
            if len(sents[0]) > max_len:
                stripped_text = sents[0]
            else:
                chosen_sents = []
                separator = ' '
                current_len = 0
                for sent in sents:
                    current_len += len(sent)
                    if current_len < max_len:
                        chosen_sents.append(sent)
                        current_len += len(separator)
                    else:
                        break
                stripped_text = separator.join(chosen_sents)
        return stripped_text
    
    def __make_text_end_with(self, text, tag):
        if not text.strip().endswith(tag):
            text = self.separator.join([text, tag])
        return text
    
    def __clean_and_save_button_texts(self, session_id, generated_buttons_steps):
        buttons_plain_text = generated_buttons_steps.split(self.__end_quest_tag)[0]
        buttons_plain_text = generated_buttons_steps.split(self.__start_fragment_tag)[0]
        buttons_plain_text = generated_buttons_steps.split(self.__input_tag)[0]
       
         
        button_texts = []
        if self.__button_tag in buttons_plain_text:
            button_texts = [self.__strip_text(button_text, self.max_button_len) for button_text in buttons_plain_text.split(self.__button_tag) if button_text.strip()]
        #Если только одна кнопка без тэга окончания
        if not button_texts:
            button_texts = [self.__strip_text(buttons_plain_text, self.max_button_len)]
        #ToDo Если не было тэга инпута, проверять, что последняя кнопка является полноценным текстом
        self.__save_button_texts(session_id, button_texts)
        return button_texts
    
    def __save_button_texts(self, session_id, button_texts):
        button_fragments = [self.separator.join([self.__button_tag, button_text]) for button_text in button_texts]
        new_fragment = self.separator.join(button_fragments)
        self.__files_organizer.add_next_part(session_id, new_fragment)
        

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
quest_generator = QuestGenerator(gpt_model, tokenizer, QUEST_DIRECTORY, stories) 

In [ ]:
def inform_about_long_operation(chat_id):
    try:
        bot.send_message(chat_id, 'Пожалуйста, подождите', reply_markup=telebot.types.ReplyKeyboardRemove())
    except Exception as e:
        print(e)

def report_about_error(chat_id):
    try:
        bot.send_message(chat_id, 'Произошла непредвиденная ошибка')
    except Exception as e:
        print(е)
        print('Сообщение об ошибке не отправлено')

@bot.message_handler(commands=['help'])
def help_message(message):
    try:
        bot.send_message(message.chat.id, help_message_text, reply_markup=telebot.types.ReplyKeyboardRemove())
    except Exception as e:
        print(e)
        report_about_error(message.chat.id)
    
@bot.message_handler(commands=['start'])
def start_message(message):
    try:
        quest_generator.new_quest(message.chat.id)
        suggested_beginning = quest_generator.suggest_beginning()
        keyboard = telebot.types.ReplyKeyboardMarkup(True, one_time_keyboard=True)
        keyboard.row(suggested_beginning)
        bot.send_message(message.chat.id, start_message_text, reply_markup=keyboard)
        bot.register_next_step_handler(message, process_first_fragment)
    except Exception as e:
        print(e)
        report_about_error(message.chat.id)

def check_buttons(buttons):
    buttons = [button for button in buttons if re.search('[А-ЯЁа-яёA-Za-z]', button)]
    if not buttons:
        buttons = ['Далее']
    return buttons
        
def process_first_fragment(message):
    try:
        inform_about_long_operation(message.chat.id)
        text = message.text
        quest_generator.save_first_fragment(message.chat.id, text)
        buttons = check_buttons(quest_generator.generate_buttons(message.chat.id))
        keyboard = telebot.types.ReplyKeyboardMarkup(True, one_time_keyboard=True)
        keyboard.row(*buttons)
        #keyboard = telebot.types.InlineKeyboardMarkup()
        #for b in buttons:
        #   print(b)
        #   keyboard.add(telebot.types.InlineKeyboardButton(b,callback_data=b[:30]))
        bot.send_message(message.chat.id, 'Выберите одну из кнопок или введите свой вариант', reply_markup=keyboard)
    except Exception as e:
        print(e)
        report_about_error(message.chat.id)

@bot.message_handler(content_types=['text'])
def user_input(message):
    try:
    ##ToDo обрабатывать случаи, когда пользователь отправляет сообщение раньше, чем получил ответ на предыдущее
        if message.text == suggest_new_quest_button_text:
            start_message(message)
        else:
            inform_about_long_operation(message.chat.id)
            quest_generator.save_input(message.chat.id, message.text)
            fragment_text, buttons, is_ending = quest_generator.generate_fragment(message.chat.id)
            if not fragment_text.strip():
                fragment_text = 'Простите, бот не смог сгенерировать текст'
            if not buttons and not is_ending:
                buttons = quest_generator.generate_buttons(message.chat.id)  
            buttons = check_buttons(buttons)
            keyboard = telebot.types.ReplyKeyboardMarkup(True, one_time_keyboard=True)
            keyboard.row(*buttons)
            #keyboard = telebot.types.InlineKeyboardMarkup()
            #for b in buttons:
            #    print(b)
            #    keyboard.add(telebot.types.InlineKeyboardButton(b,callback_data=b[:30]))
            bot.send_message(message.chat.id, fragment_text, reply_markup=keyboard)
    except Exception as e:
        print(e)
        report_about_error(message.chat.id)

if __name__ == '__main__':
    bot.polling()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


## Примеры работы выбранной формулы ранжирования

In [ ]:
def get_fixed_part_coefs(fixed_part):
    input_idx = tokenizer.encode(fixed_part, return_tensors='pt').cuda()
    loss = gpt_model(input_idx, labels=input_idx)[0]
    return loss.exp(), input_idx.shape[-1]

In [ ]:
def get_quest_data_fragment(previous_fragment, next_text_candidate):
    return '\n'.join(['BOF', previous_fragment, 'BUTTON', 'Далее', 'INPUT', 'Далее', 'BOF', next_text_candidate])

In [ ]:
def calc_perplexity_based_score(text, fixed_part_perplexity, fixed_part_len):    
    input_idx = tokenizer.encode(text, return_tensors='pt').cuda()
    loss = gpt_model(input_idx, labels=input_idx)[0]
    return (loss.exp()-fixed_part_perplexity)/(input_idx.shape[-1]-fixed_part_len)

In [ ]:
human_written_fragments = ['Вы выходите на поляну и видите недостроенный стул из снега, возле которого лежит грустный плюшевый мишка. Хотите достроить?',
  'Вы подошли к выходу из леса и еще можете успеть на последний автобус в город. Но вы так и не выяснили, что произошло с ребенком. Что будете делать?',
  'Вы выходите к поляне, надеетесь, что идти по ней будет легче. Делаете шаг вперед и проваливаетесь по пояс. Развернетесь?',
 'Вы слышите тот же детский голос снова. Но теперь ребенок не кричит, а репетирует «Я помню чудное мгновенье». Но почему он делает это в лесу?',
  'Вы обнаруживаете следы на снегу, размер ноги определенно не как у ребенка. Посмотрите, куда они ведут.',
 'Вы совсем забыли, откуда пришли. Может, найти дорогу назад важнее, чем понять, откуда доносились звуки?',
  'Теперь голос доносится сверху, причем очень близко. Под вашими ногами валяется ботинок, ребенок сидит наверху и не хочет слезать в сугроб. Вы подаете ему обувь, он благодарит вас и говорит, что слезет, когда захочет.',
 'Между деревьев вы обнаруживаете колонку, из которой доносится тот же детский крик. Вы выключаете ее, чтобы звуки больше никого не сбили с толку.',
                          'Пора сразиться с чудовищем']

In [ ]:
previous_fragments_to_score = [
    'Вас все достало и вы ушли домой. Сколько можно находиться в этом лесу!',
    'Вам очень интересно, откуда доносятся звуки. Вы тут, чтобы узнать это, и не хотите отвлекаться.',
    'Вы уверены, что ничего страшного не произошло. Ребенок никуда не денется',
    'Вы нашли труп',
    'Лизу убьет научник',
    'Лизу убьет руководитель практики',
    'На вас напало пятнадцать единорогов',
    'Кошки пушистые',
    'Петя купил творог для сырников', 
    'Вы решили приступить к сражению'
]

In [ ]:
import torch

In [ ]:
scores = {}
gpt_model.eval()
with torch.no_grad():
    for i, next_fragment in enumerate(human_written_fragments):
        current_fragment_scores = []
    
        fixed_part = '\n'.join(['BUTTON', 'Далее', 'INPUT', 'Далее', 'BOF', next_fragment])
        fixed_part_loss, fixed_part_len = get_fixed_part_coefs(fixed_part)
    
        for previous_fragment in previous_fragments_to_score:
            joined_fragment_to_score = get_quest_data_fragment(previous_fragment, next_fragment)
            perplexity_based_score = calc_perplexity_based_score(joined_fragment_to_score, fixed_part_loss, fixed_part_len)
            current_fragment_scores.append({'previous_fragment':previous_fragment, 'score':perplexity_based_score})
        current_fragment_scores.sort(key=lambda x: x['score'])
        scores[next_fragment] = current_fragment_scores

In [ ]:
for next_fragment, next_fragment_scores in scores.items():
    print(next_fragment)
    for score_data in next_fragment_scores:
        print(score_data['previous_fragment'], score_data['score'])
    print('__________________________')

Вы выходите на поляну и видите недостроенный стул из снега, возле которого лежит грустный плюшевый мишка. Хотите достроить?
Петя купил творог для сырников tensor(3.0162, device='cuda:0')
Вы уверены, что ничего страшного не произошло. Ребенок никуда не денется tensor(3.0323, device='cuda:0')
Вы решили приступить к сражению tensor(3.3744, device='cuda:0')
Вы нашли труп tensor(5.7282, device='cuda:0')
Кошки пушистые tensor(6.9726, device='cuda:0')
Вам очень интересно, откуда доносятся звуки. Вы тут, чтобы узнать это, и не хотите отвлекаться. tensor(8.5286, device='cuda:0')
На вас напало пятнадцать единорогов tensor(10.2027, device='cuda:0')
Вас все достало и вы ушли домой. Сколько можно находиться в этом лесу! tensor(13.2297, device='cuda:0')
Лизу убьет руководитель практики tensor(27.6494, device='cuda:0')
Лизу убьет научник tensor(30.2868, device='cuda:0')
__________________________
Вы подошли к выходу из леса и еще можете успеть на последний автобус в город. Но вы так и не выяснили, чт

In [ ]:
i

0